In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
input_path = ''

application_train = pd.read_csv(input_path+'application_train.csv')
application_test = pd.read_csv(input_path+'application_test.csv')
bureau = pd.read_csv(input_path+'bureau.csv')
bureau_balance = pd.read_csv(input_path+'bureau_balance.csv')
previous_application = pd.read_csv(input_path+'previous_application.csv')
POS_CASH_balance = pd.read_csv(input_path+'POS_CASH_balance.csv')
installments_payments = pd.read_csv(input_path+'installments_payments.csv')
credit_card_balance = pd.read_csv(input_path+'credit_card_balance.csv')

print('Size of application_train data', application_train.shape)
print('Size of application_test data', application_test.shape)
print('Size of bureau data', bureau.shape)
print('Size of bureau_balance data', bureau_balance.shape)
print('Size of previous_application data', previous_application.shape)
print('Size of POS_CASH_balance data', POS_CASH_balance.shape)
print('Size of installments_payments data', installments_payments.shape)
print('Size of credit_card_balance data', credit_card_balance.shape)

Size of application_train data (307511, 122)
Size of application_test data (48744, 121)
Size of bureau data (1716428, 17)
Size of bureau_balance data (27299925, 3)
Size of previous_application data (1670214, 37)
Size of POS_CASH_balance data (10001358, 8)
Size of installments_payments data (13605401, 8)
Size of credit_card_balance data (3840312, 23)


### Missing Values
Now let's ahve a look at the empty values in columns.

In [2]:
# Calculate missing values by column,
# if missing values consist of more than 'threshold' of columns, we just drop this column
def missing_values_table(df,threshold = 0.6):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        #pass through threshold
        missing_cols = list(mis_val_table.index[mis_val_table['% of Total Values'] > threshold*100])
        mis_val_table_filter = mis_val_table.drop(index=missing_cols)
        df_filter = df.drop(columns = missing_cols)
        # Sort the table by percentage of missing descending
        mis_val_table = mis_val_table[
            mis_val_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table.shape[0]) +
              " columns that have missing values.")
        print(f'After filtering, {len(missing_cols)} cols are removed, now there are {mis_val_table_filter.shape[0]} cols left ')
        
        # Return the dataframe with missing information
        return df_filter,missing_cols

In [3]:
missing_col_list = []
print('Missing value on application_train')
application_train_filter,application_train_filter_list = missing_values_table(application_train)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on application_test')
application_test_filter,application_train_filter_list = missing_values_table(application_test)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on bureau')
bureau_filter,application_train_filter_list = missing_values_table(bureau)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on bureau_balance')
bureau_balance_filter,application_train_filter_list = missing_values_table(bureau_balance)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on previous_application')
previous_application_filter,application_train_filter_list = missing_values_table(previous_application)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on POS_CASH_balance')
POS_CASH_balance_filter,application_train_filter_list = missing_values_table(POS_CASH_balance)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on installments_payments')
installments_payments_filter,application_train_filter_list = missing_values_table(installments_payments)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on credit_card_balance')
credit_card_balance_filter,application_train_filter_list = missing_values_table(credit_card_balance)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

Missing value on application_train
Your selected dataframe has 122 columns.
There are 67 columns that have missing values.
After filtering, 17 cols are removed, now there are 105 cols left 
Missing value on application_test
Your selected dataframe has 121 columns.
There are 64 columns that have missing values.
After filtering, 17 cols are removed, now there are 104 cols left 
Missing value on bureau
Your selected dataframe has 17 columns.
There are 7 columns that have missing values.
After filtering, 2 cols are removed, now there are 15 cols left 
Missing value on bureau_balance
Your selected dataframe has 3 columns.
There are 0 columns that have missing values.
After filtering, 0 cols are removed, now there are 3 cols left 
Missing value on previous_application
Your selected dataframe has 37 columns.
There are 16 columns that have missing values.
After filtering, 2 cols are removed, now there are 35 cols left 
Missing value on POS_CASH_balance
Your selected dataframe has 8 columns.
Th

In [4]:
print(121+17+3+37+8+8+23)
print(67+7+16+2+2+9)
print(103/217)
missing_col_list = list(set(missing_col_list))
print(len(missing_col_list))

217
103
0.47465437788018433
21


### Check feature importance

In [5]:
correlations = application_train.corr()['TARGET'].sort_values()
print('Most Positive Correlations:\n', correlations.tail(10))
print('\nMost Negative Correlations:\n', correlations.head(10))

Most Positive Correlations:
 FLAG_DOCUMENT_3                0.044346
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645
REGION_POPULATION_RELATIVE   -0.037227
ELEVATORS_AVG                -0.034199
Name: TARGET, dtype: float64


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Data aggregation

In [14]:
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))
            
# Assign the list of columns names as the dataframe column names
bureau_agg.columns = columns
#bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [15]:
# Merge with the training data
train = application_train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,8.0,-499.875,-7.0,-1185.0,-3999.0,7.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,4.0,-816.000,-43.0,-2131.0,-3264.0,0.0,NaN,NaN,NaN,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,2.0,-532.000,-382.0,-682.0,-1064.0,0.0,NaN,NaN,NaN,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,1.0,-783.000,-783.0,-783.0,-783.0,0.0,NaN,NaN,NaN,0.0


In [17]:

# review correlations
new_cor_list = []

# Iterate through the columns 
for col in columns:
    # Calculate correlation with the target
    corr = train['TARGET'].corr(train[col])
    
    # Append the list as a tuple

    new_cor_list.append((col, corr))
    
# Sort the correlations by the absolute value
# Make sure to reverse to put the largest values at the front of list
new_cor_list = sorted(new_cor_list, key = lambda x: abs(x[1]), reverse = True)
new_cor_list[:15]

[('bureau_DAYS_CREDIT_mean', 0.08972896721998116),
 ('bureau_DAYS_CREDIT_min', 0.0752482510301037),
 ('bureau_DAYS_CREDIT_UPDATE_mean', 0.06892735266968673),
 ('bureau_DAYS_ENDDATE_FACT_min', 0.05588737984392085),
 ('bureau_DAYS_CREDIT_ENDDATE_sum', 0.053734895601020544),
 ('bureau_DAYS_ENDDATE_FACT_mean', 0.053199625857586336),
 ('bureau_DAYS_CREDIT_max', 0.049782054639973074),
 ('bureau_DAYS_ENDDATE_FACT_sum', 0.04885350261111597),
 ('bureau_DAYS_CREDIT_ENDDATE_mean', 0.046982754334835404),
 ('bureau_DAYS_CREDIT_UPDATE_min', 0.042863922470730204),
 ('bureau_DAYS_CREDIT_sum', 0.041999824814846765),
 ('bureau_DAYS_CREDIT_UPDATE_sum', 0.04140363535306015),
 ('bureau_DAYS_CREDIT_ENDDATE_max', 0.03658963469632896),
 ('bureau_DAYS_CREDIT_ENDDATE_min', 0.034281109921615996),
 ('bureau_DAYS_ENDDATE_FACT_count', -0.03049230665332553)]

### Categorical Variables

In [7]:
#categorical = pd.get_dummies(bureau.select_dtypes('object'))
#categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
#print(len(categorical))
#print(len(bureau['SK_ID_CURR']))
#categorical.head(10)
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [29]:
print(f'train.shape:{train.shape}')
#print(f'categorical_grouped.shape:{train.shape}')
categorical_grouped = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
train = train.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
print(f'train.shape:{train.shape}')
categorical_grouped.head()

train.shape:(307511, 182)
train.shape:(307511, 228)


,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


### Aggregation

In [8]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make  new features
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [9]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_new.head()

/tmp/ipykernel_18778/180609655.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [15]:
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
#Ebureau.columns

/tmp/ipykernel_18778/180609655.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


In [19]:
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
# Merge to include the SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
bureau_by_loan.head()

,SK_ID_BUREAU,SK_ID_CURR,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,...,bureau_balance_STATUS_3_count,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm
0,5714462,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5714463,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5714464,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5714465,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5714466,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
bureau_balance_by_client.head()

,SK_ID_CURR,client_bureau_balance_MONTHS_BALANCE_count_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_count_sum,client_bureau_balance_MONTHS_BALANCE_mean_count,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_mean_max,client_bureau_balance_MONTHS_BALANCE_mean_min,...,client_bureau_balance_STATUS_X_count_count,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_X_count_sum,client_bureau_balance_STATUS_X_count_norm_count,client_bureau_balance_STATUS_X_count_norm_mean,client_bureau_balance_STATUS_X_count_norm_max,client_bureau_balance_STATUS_X_count_norm_min,client_bureau_balance_STATUS_X_count_norm_sum
0,100001,7,24.571429,52.0,2.0,172.0,7,-11.785714,-0.5,-25.5,...,7,4.285714,9.0,0.0,30.0,7,0.214590,0.500000,0.0,1.502129
1,100002,8,13.750000,22.0,4.0,110.0,8,-21.875000,-1.5,-39.5,...,8,1.875000,3.0,0.0,15.0,8,0.161932,0.500000,0.0,1.295455
2,100003,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
3,100004,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
4,100005,3,7.000000,13.0,3.0,21.0,3,-3.000000,-1.0,-6.0,...,3,0.666667,1.0,0.0,2.0,3,0.136752,0.333333,0.0,0.410256


In [21]:
# Merge with the value counts of bureau
train = application_train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [22]:
new_features = list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))
print(train.shape)

Number of features using previous loans from other institutions data:  333
(307511, 333)


In [30]:
train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
#print(train_filter_list)

Your selected dataframe has 333 columns.
There are 278 columns that have missing values.
After filtering, 66 cols are removed, now there are 267 cols left 


In [31]:
print(train_filter_list)
print(len(train_filter_list))

['bureau_AMT_ANNUITY_mean', 'bureau_AMT_ANNUITY_max', 'bureau_AMT_ANNUITY_min', 'client_bureau_balance_MONTHS_BALANCE_count_mean', 'client_bureau_balance_MONTHS_BALANCE_count_max', 'client_bureau_balance_MONTHS_BALANCE_count_min', 'client_bureau_balance_MONTHS_BALANCE_mean_mean', 'client_bureau_balance_MONTHS_BALANCE_mean_max', 'client_bureau_balance_MONTHS_BALANCE_mean_min', 'client_bureau_balance_MONTHS_BALANCE_max_mean', 'client_bureau_balance_MONTHS_BALANCE_max_max', 'client_bureau_balance_MONTHS_BALANCE_max_min', 'client_bureau_balance_MONTHS_BALANCE_min_mean', 'client_bureau_balance_MONTHS_BALANCE_min_max', 'client_bureau_balance_MONTHS_BALANCE_min_min', 'client_bureau_balance_MONTHS_BALANCE_sum_mean', 'client_bureau_balance_MONTHS_BALANCE_sum_max', 'client_bureau_balance_MONTHS_BALANCE_sum_min', 'client_bureau_balance_STATUS_0_count_mean', 'client_bureau_balance_STATUS_0_count_max', 'client_bureau_balance_STATUS_0_count_min', 'client_bureau_balance_STATUS_0_count_norm_mean', 'cl

In [32]:
# Merge on testing data
test = application_test.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [40]:
#train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
test_filter,test_filter_list = missing_values_table(test,threshold = 0.7)
test_filter_list.extend(train_filter_list)
filter_list = list(set(test_filter_list))
print(f'filter_list shape:{len(test_filter_list)}')
test_filter = test.drop(columns = filter_list)
train_filter = train.drop(columns = filter_list)
print(f'train_filter shape:{train_filter.shape}')
print(f'test_filter shape:{test_filter.shape}')

Your selected dataframe has 332 columns.
There are 275 columns that have missing values.
After filtering, 0 cols are removed, now there are 332 cols left 
filter_list shape:66
train_filter shape:(307511, 267)
test_filter shape:(48744, 266)


In [49]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import gc


In [50]:
def model(features, test_features, n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding

    features = pd.get_dummies(features)
    test_features = pd.get_dummies(test_features)

    # Align the dataframes by the columns
    features, test_features = features.align(test_features, join = 'inner', axis = 1)

    # No categorical indices to record
    cat_indices = 'auto'
    
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [51]:
submission, fi, metrics = model(train, test)

Training Data Shape:  (307511, 452)
Testing Data Shape:  (48744, 452)


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.810821	train's binary_logloss: 0.53617	valid's auc: 0.763279	valid's binary_logloss: 0.553134


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.810727	train's binary_logloss: 0.53666	valid's auc: 0.765248	valid's binary_logloss: 0.553447


...

[200]	train's auc: 0.809569	train's binary_logloss: 0.538079	valid's auc: 0.770517	valid's binary_logloss: 0.55476


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.810367	train's binary_logloss: 0.537096	valid's auc: 0.765831	valid's binary_logloss: 0.552574


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.810433	train's binary_logloss: 0.536726	valid's auc: 0.76417	valid's binary_logloss: 0.554751


In [52]:
metrics

,fold,train,valid
0,0,0.823660,0.763730
1,1,0.824682,0.766092
2,2,0.826798,0.771361
3,3,0.817720,0.766206
4,4,0.821525,0.764642
5,overall,0.822877,0.766404


### Pred on bureau only

In [53]:
submission.to_csv('pred_kaggle.csv', index = False)

In [54]:
previous_agg = agg_numeric(previous_application, 'SK_ID_CURR', 'previous')
print('Previous aggregation shape: ', previous_agg.shape)
previous_agg.head()

/tmp/ipykernel_18778/180609655.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


Previous aggregation shape:  (338857, 96)


,SK_ID_CURR,previous_AMT_ANNUITY_count,previous_AMT_ANNUITY_mean,previous_AMT_ANNUITY_max,previous_AMT_ANNUITY_min,previous_AMT_ANNUITY_sum,previous_AMT_APPLICATION_count,previous_AMT_APPLICATION_mean,previous_AMT_APPLICATION_max,previous_AMT_APPLICATION_min,...,previous_DAYS_TERMINATION_count,previous_DAYS_TERMINATION_mean,previous_DAYS_TERMINATION_max,previous_DAYS_TERMINATION_min,previous_DAYS_TERMINATION_sum,previous_NFLAG_INSURED_ON_APPROVAL_count,previous_NFLAG_INSURED_ON_APPROVAL_mean,previous_NFLAG_INSURED_ON_APPROVAL_max,previous_NFLAG_INSURED_ON_APPROVAL_min,previous_NFLAG_INSURED_ON_APPROVAL_sum
0,100001,1,3951.000,3951.000,3951.000,3951.000,1,24835.50,24835.5,24835.5,...,1,-1612.000000,-1612.0,-1612.0,-1612.0,1,0.000000,0.0,0.0,0.0
1,100002,1,9251.775,9251.775,9251.775,9251.775,1,179055.00,179055.0,179055.0,...,1,-17.000000,-17.0,-17.0,-17.0,1,0.000000,0.0,0.0,0.0
2,100003,3,56553.990,98356.995,6737.310,169661.970,3,435436.50,900000.0,68809.5,...,3,-1047.333333,-527.0,-1976.0,-3142.0,3,0.666667,1.0,0.0,2.0
3,100004,1,5357.250,5357.250,5357.250,5357.250,1,24282.00,24282.0,24282.0,...,1,-714.000000,-714.0,-714.0,-714.0,1,0.000000,0.0,0.0,0.0
4,100005,1,4813.200,4813.200,4813.200,4813.200,2,22308.75,44617.5,0.0,...,1,-460.000000,-460.0,-460.0,-460.0,1,0.000000,0.0,0.0,0.0


In [57]:
previous_counts = count_categorical(previous_application, 'SK_ID_CURR', 'previous')
# Merge in the previous information
# train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
# train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

# test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
# test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

In [59]:
train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

In [60]:
train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
test_filter,test_filter_list = missing_values_table(test,threshold = 0.7)
test_filter_list.extend(train_filter_list)
filter_list = list(set(test_filter_list))
print(f'filter_list shape:{len(test_filter_list)}')
test_filter = test.drop(columns = filter_list)
train_filter = train.drop(columns = filter_list)
print(f'train_filter shape:{train_filter.shape}')
print(f'test_filter shape:{test_filter.shape}')

Your selected dataframe has 1095 columns.
There are 1040 columns that have missing values.
After filtering, 78 cols are removed, now there are 1017 cols left 
Your selected dataframe has 1094 columns.
There are 1037 columns that have missing values.
After filtering, 12 cols are removed, now there are 1082 cols left 
filter_list shape:90
train_filter shape:(307511, 1017)
test_filter shape:(48744, 1016)


In [65]:
def aggregate_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    
    # Aggregate the numeric columns
    df_agg = agg_numeric(df,group_vars[0],df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'category'):
    
        # Count the categorical columns
        df_counts = count_categorical(df, parent_var = group_vars[0], df_name = df_names[0])

        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan, group_vars[1],df_names[1])

        
    # No categorical variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = agg_numeric(df_by_loan,  group_vars[1],  df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

In [66]:
cash_by_client = aggregate_client(POS_CASH_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
cash_by_client.head()

/tmp/ipykernel_18778/180609655.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


,SK_ID_CURR,client_cash_MONTHS_BALANCE_count_count,client_cash_MONTHS_BALANCE_count_mean,client_cash_MONTHS_BALANCE_count_max,client_cash_MONTHS_BALANCE_count_min,client_cash_MONTHS_BALANCE_count_sum,client_cash_MONTHS_BALANCE_mean_count,client_cash_MONTHS_BALANCE_mean_mean,client_cash_MONTHS_BALANCE_mean_max,client_cash_MONTHS_BALANCE_mean_min,...,client_cash_SK_DPD_DEF_min_count,client_cash_SK_DPD_DEF_min_mean,client_cash_SK_DPD_DEF_min_max,client_cash_SK_DPD_DEF_min_min,client_cash_SK_DPD_DEF_min_sum,client_cash_SK_DPD_DEF_sum_count,client_cash_SK_DPD_DEF_sum_mean,client_cash_SK_DPD_DEF_sum_max,client_cash_SK_DPD_DEF_sum_min,client_cash_SK_DPD_DEF_sum_sum
0,100001,9,4.555556,5,4,41,9,-72.555556,-55.0,-94.5,...,9,0.0,0,0,0,9,3.111111,7,0,28
1,100002,19,19.000000,19,19,361,19,-10.000000,-10.0,-10.0,...,19,0.0,0,0,0,19,0.000000,0,0,0
2,100003,28,9.714286,12,8,272,28,-43.785714,-21.5,-71.5,...,28,0.0,0,0,0,28,0.000000,0,0,0
3,100004,4,4.000000,4,4,16,4,-25.500000,-25.5,-25.5,...,4,0.0,0,0,0,4,0.000000,0,0,0
4,100005,11,11.000000,11,11,121,11,-20.000000,-20.0,-20.0,...,11,0.0,0,0,0,11,0.000000,0,0,0


In [67]:
train = train_filter
test = test_filter
train = train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')

train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
test_filter,test_filter_list = missing_values_table(test,threshold = 0.7)
test_filter_list.extend(train_filter_list)
filter_list = list(set(test_filter_list))
print(f'filter_list shape:{len(test_filter_list)}')
test_filter = test.drop(columns = filter_list)
train_filter = train.drop(columns = filter_list)
print(f'train_filter shape:{train_filter.shape}')
print(f'test_filter shape:{test_filter.shape}')

Your selected dataframe has 1142 columns.
There are 1087 columns that have missing values.
After filtering, 0 cols are removed, now there are 1142 cols left 
Your selected dataframe has 1141 columns.
There are 1084 columns that have missing values.
After filtering, 0 cols are removed, now there are 1141 cols left 
filter_list shape:0
train_filter shape:(307511, 1142)
test_filter shape:(48744, 1141)


In [69]:
installments_by_client = aggregate_client(installments_payments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
installments_by_client.head()

,SK_ID_CURR,client_installments_NUM_INSTALMENT_VERSION_count_count,client_installments_NUM_INSTALMENT_VERSION_count_mean,client_installments_NUM_INSTALMENT_VERSION_count_max,client_installments_NUM_INSTALMENT_VERSION_count_min,client_installments_NUM_INSTALMENT_VERSION_count_sum,client_installments_NUM_INSTALMENT_VERSION_mean_count,client_installments_NUM_INSTALMENT_VERSION_mean_mean,client_installments_NUM_INSTALMENT_VERSION_mean_max,client_installments_NUM_INSTALMENT_VERSION_mean_min,...,client_installments_AMT_PAYMENT_min_count,client_installments_AMT_PAYMENT_min_mean,client_installments_AMT_PAYMENT_min_max,client_installments_AMT_PAYMENT_min_min,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_PAYMENT_sum_count,client_installments_AMT_PAYMENT_sum_mean,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_PAYMENT_sum_min,client_installments_AMT_PAYMENT_sum_sum
0,100001,7,3.571429,4,3,25,7,1.142857,1.250000,1.000000,...,7,3963.8250,3980.925,3951.000,27746.775,7,21834.096429,29250.900,11945.025,1.528387e+05
1,100002,19,19.000000,19,19,361,19,1.052632,1.052632,1.052632,...,19,9251.7750,9251.775,9251.775,175783.725,19,219625.695000,219625.695,219625.695,4.172888e+06
2,100003,25,9.160000,12,6,229,25,1.040000,1.142857,1.000000,...,25,46164.3318,98356.995,6662.970,1154108.295,25,453952.220400,1150977.330,80773.380,1.134881e+07
3,100004,3,3.000000,3,3,9,3,1.333333,1.333333,1.333333,...,3,5357.2500,5357.250,5357.250,16071.750,3,21288.465000,21288.465,21288.465,6.386540e+04
4,100005,9,9.000000,9,9,81,9,1.111111,1.111111,1.111111,...,9,4813.2000,4813.200,4813.200,43318.800,9,56161.845000,56161.845,56161.845,5.054566e+05


In [70]:
train = train_filter
test = test_filter
train = train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')

train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
test_filter,test_filter_list = missing_values_table(test,threshold = 0.7)
test_filter_list.extend(train_filter_list)
filter_list = list(set(test_filter_list))
print(f'filter_list shape:{len(test_filter_list)}')
test_filter = test.drop(columns = filter_list)
train_filter = train.drop(columns = filter_list)
print(f'train_filter shape:{train_filter.shape}')
print(f'test_filter shape:{test_filter.shape}')

Your selected dataframe has 1292 columns.
There are 1237 columns that have missing values.
After filtering, 0 cols are removed, now there are 1292 cols left 
Your selected dataframe has 1291 columns.
There are 1234 columns that have missing values.
After filtering, 0 cols are removed, now there are 1291 cols left 
filter_list shape:0
train_filter shape:(307511, 1292)
test_filter shape:(48744, 1291)


In [71]:
credit_by_client = aggregate_client(credit_card_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
credit_by_client.head()

/tmp/ipykernel_18778/180609655.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


,SK_ID_CURR,client_credit_MONTHS_BALANCE_count_count,client_credit_MONTHS_BALANCE_count_mean,client_credit_MONTHS_BALANCE_count_max,client_credit_MONTHS_BALANCE_count_min,client_credit_MONTHS_BALANCE_count_sum,client_credit_MONTHS_BALANCE_mean_count,client_credit_MONTHS_BALANCE_mean_mean,client_credit_MONTHS_BALANCE_mean_max,client_credit_MONTHS_BALANCE_mean_min,...,client_credit_SK_DPD_DEF_min_count,client_credit_SK_DPD_DEF_min_mean,client_credit_SK_DPD_DEF_min_max,client_credit_SK_DPD_DEF_min_min,client_credit_SK_DPD_DEF_min_sum,client_credit_SK_DPD_DEF_sum_count,client_credit_SK_DPD_DEF_sum_mean,client_credit_SK_DPD_DEF_sum_max,client_credit_SK_DPD_DEF_sum_min,client_credit_SK_DPD_DEF_sum_sum
0,100006,6,6.0,6,6,36,6,-3.5,-3.5,-3.5,...,6,0.0,0,0,0,6,0.0,0,0,0
1,100011,74,74.0,74,74,5476,74,-38.5,-38.5,-38.5,...,74,0.0,0,0,0,74,0.0,0,0,0
2,100013,96,96.0,96,96,9216,96,-48.5,-48.5,-48.5,...,96,0.0,0,0,0,96,1.0,1,1,96
3,100021,17,17.0,17,17,289,17,-10.0,-10.0,-10.0,...,17,0.0,0,0,0,17,0.0,0,0,0
4,100023,8,8.0,8,8,64,8,-7.5,-7.5,-7.5,...,8,0.0,0,0,0,8,0.0,0,0,0


In [72]:
train = train_filter
test = test_filter
train = train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
test = test.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')

train_filter,train_filter_list = missing_values_table(train,threshold = 0.7)
test_filter,test_filter_list = missing_values_table(test,threshold = 0.7)
test_filter_list.extend(train_filter_list)
filter_list = list(set(test_filter_list))
print(f'filter_list shape:{len(test_filter_list)}')
test_filter = test.drop(columns = filter_list)
train_filter = train.drop(columns = filter_list)
print(f'train_filter shape:{train_filter.shape}')
print(f'test_filter shape:{test_filter.shape}')

Your selected dataframe has 1792 columns.
There are 1737 columns that have missing values.
After filtering, 500 cols are removed, now there are 1292 cols left 
Your selected dataframe has 1791 columns.
There are 1734 columns that have missing values.
After filtering, 63 cols are removed, now there are 1728 cols left 
filter_list shape:563
train_filter shape:(307511, 1292)
test_filter shape:(48744, 1291)


In [73]:
train = train_filter
test = test_filter


In [74]:
submission, fi, metrics = model(train, test)

Training Data Shape:  (307511, 1411)
Testing Data Shape:  (48744, 1411)


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.83067	train's binary_logloss: 0.514902	valid's auc: 0.782106	valid's binary_logloss: 0.534026
[400]	train's auc: 0.867814	train's binary_logloss: 0.473096	valid's auc: 0.782918	valid's binary_logloss: 0.50764


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.831759	train's binary_logloss: 0.513632	valid's auc: 0.779841	valid's binary_logloss: 0.533983


...

[200]	train's auc: 0.830859	train's binary_logloss: 0.514726	valid's auc: 0.785143	valid's binary_logloss: 0.535057
[400]	train's auc: 0.868786	train's binary_logloss: 0.472401	valid's auc: 0.786206	valid's binary_logloss: 0.508275


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.830749	train's binary_logloss: 0.515123	valid's auc: 0.783771	valid's binary_logloss: 0.533958
[400]	train's auc: 0.868374	train's binary_logloss: 0.473	valid's auc: 0.783978	valid's binary_logloss: 0.507973


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.830919	train's binary_logloss: 0.514578	valid's auc: 0.779881	valid's binary_logloss: 0.53593
[400]	train's auc: 0.867987	train's binary_logloss: 0.473247	valid's auc: 0.781543	valid's binary_logloss: 0.509666


In [75]:
metrics

,fold,train,valid
0,0,0.872299,0.783065
1,1,0.845723,0.780889
2,2,0.854530,0.786422
3,3,0.853833,0.784470
4,4,0.871910,0.781814
5,overall,0.859659,0.783211


In [76]:
submission.to_csv('pred_kaggle_all_gbm.csv', index = False)

In [77]:
correlations = train.corr()['TARGET'].sort_values()
print('Most Positive Correlations:\n', correlations.tail(10))
print('\nMost Negative Correlations:\n', correlations.head(10))

Most Positive Correlations:
 bureau_CREDIT_ACTIVE_Active_count_norm                          0.077356
previous_NAME_CONTRACT_STATUS_Refused_count_norm_x              0.077671
previous_NAME_CONTRACT_STATUS_Refused_count_norm_y              0.077671
DAYS_BIRTH                                                      0.078239
bureau_DAYS_CREDIT_mean                                         0.089729
TARGET                                                          1.000000
previous_NAME_GOODS_CATEGORY_House Construction_count_x              NaN
previous_NAME_GOODS_CATEGORY_House Construction_count_norm_x         NaN
previous_NAME_GOODS_CATEGORY_House Construction_count_y              NaN
previous_NAME_GOODS_CATEGORY_House Construction_count_norm_y         NaN
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                                          -0.178919
EXT_SOURCE_2                                          -0.160472
EXT_SOURCE_1                                          

### For train_app only

In [84]:
missing_col_list = []
print('Missing value on application_train')
application_train_filter,application_train_filter_list = missing_values_table(application_train)
missing_col_list.extend(application_train_filter_list)
print('=='*20)

print('Missing value on application_test')
application_test_filter,application_test_filter_list = missing_values_table(application_test)
application_test_filter_list.extend(application_train_filter_list)
print('=='*20)

Missing value on application_train
Your selected dataframe has 122 columns.
There are 67 columns that have missing values.
After filtering, 17 cols are removed, now there are 105 cols left 
Missing value on application_test
Your selected dataframe has 121 columns.
There are 64 columns that have missing values.
After filtering, 17 cols are removed, now there are 104 cols left 


In [86]:
application_test_filter_list = list(set(application_test_filter_list))
application_train.drop(columns =application_test_filter_list)
application_test.drop(columns =application_test_filter_list)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [87]:
submission, fi, metrics = model(application_train, application_test)

Training Data Shape:  (307511, 241)
Testing Data Shape:  (48744, 241)


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.7989	train's binary_logloss: 0.547642	valid's auc: 0.755463	valid's binary_logloss: 0.563361
[400]	train's auc: 0.82864	train's binary_logloss: 0.518235	valid's auc: 0.755594	valid's binary_logloss: 0.544951


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.798638	train's binary_logloss: 0.547974	valid's auc: 0.758354	valid's binary_logloss: 0.56326


...

[200]	train's auc: 0.7977	train's binary_logloss: 0.549358	valid's auc: 0.763287	valid's binary_logloss: 0.564505


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.798947	train's binary_logloss: 0.547854	valid's auc: 0.757823	valid's binary_logloss: 0.562315


/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/hlibt/anaconda3/envs/pytorch/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	train's auc: 0.798357	train's binary_logloss: 0.548311	valid's auc: 0.758237	valid's binary_logloss: 0.564466


In [88]:
metrics

,fold,train,valid
0,0,0.815791,0.755755
1,1,0.811912,0.758533
2,2,0.811252,0.763822
3,3,0.805899,0.758345
4,4,0.807459,0.758535
5,overall,0.810463,0.759002


In [89]:
submission.to_csv('pred_kaggle_train_only.csv', index = False)

In [94]:
correlations_abs = np.abs(correlations).sort_values()
print('Most significant Correlations:\n', correlations_abs.tail(25))

Most significant Correlations:
 previous_NAME_PRODUCT_TYPE_walk-in_count_y                      0.062628
previous_NAME_PRODUCT_TYPE_walk-in_count_x                      0.062628
previous_NAME_CONTRACT_STATUS_Approved_count_norm_y             0.063521
previous_NAME_CONTRACT_STATUS_Approved_count_norm_x             0.063521
previous_NAME_CONTRACT_STATUS_Refused_count_x                   0.064469
previous_NAME_CONTRACT_STATUS_Refused_count_y                   0.064469
bureau_CREDIT_ACTIVE_Active_count                               0.067128
bureau_DAYS_CREDIT_UPDATE_mean                                  0.068927
previous_CODE_REJECT_REASON_XAP_count_norm_x                    0.073930
previous_CODE_REJECT_REASON_XAP_count_norm_y                    0.073930
bureau_DAYS_CREDIT_min                                          0.075248
bureau_CREDIT_ACTIVE_Active_count_norm                          0.077356
previous_NAME_CONTRACT_STATUS_Refused_count_norm_y              0.077671
previous_NAME_CONTR